In [1]:
import pandas as pd
from haversine import haversine, Unit
import cv2

In [2]:
def find_tile(x_pix, y_pix, x_tiles = 9, y_tiles = 5, img_size = (3072, 2048), dsize = (1856, 1392), cropped_size = (1856, 1040), tile_size = (224, 224), overlap = 20):
    x_ratio = img_size[0]/dsize[0]
    y_ratio = img_size[1]/dsize[1]
    step_x = (tile_size[0]/2) * x_ratio
    step_y = (tile_size[1]/2) * y_ratio
    y_start = img_size[1] - (img_size[1] * cropped_size[1]/dsize[1])

    x_centers = [step_x + i*(step_x*2 - overlap*x_ratio) for i in range(x_tiles)]
    y_centers = [step_y + y_start + i*(step_y*2 - overlap*y_ratio) for i in range(y_tiles)]
    
    x_scores = [abs(x_cent - x_pix) for x_cent in x_centers]
    y_scores = [abs(y_cent - y_pix) for y_cent in y_centers]
    
    return (y_scores.index(min(y_scores))*(x_tiles)) + (x_scores.index(min(x_scores)))

def split_station_dir(image_id):
    

In [6]:
cam_meta2 = pd.read_csv("../../../data/processed/camera_metadata_hpwren.csv")

In [7]:
cam_meta2

,camera_id,direction,camera_name,image_id,long,lat,elevation,geometry.type,geometry.coordinates,x_resolution_manual,y_resolution_manual,properties.description.url
0,hpwren0_unknown direction,unknown direction,NaN,tje-1-mobo-c,-117.120000,32.550000,10.0,Point,"[-117.12, 32.55, 10]",NaN,NaN,http://hpwren.ucsd.edu/cameras/TJE.html
1,hpwren1_north,north,Big Black Mountain,bm-n-mobo-c,-116.808092,33.159927,4055.0,Point,"[-116.8081, 33.1599, 4055]",3072.0,2048.0,http://hpwren.ucsd.edu/cameras/BBlackMtn.html
2,hpwren1_east,east,Big Black Mountain,bm-e-mobo-c,-116.808092,33.159927,4055.0,Point,"[-116.8081, 33.1599, 4055]",3072.0,2048.0,http://hpwren.ucsd.edu/cameras/BBlackMtn.html
3,hpwren1_south,south,Big Black Mountain,bm-s-mobo-c,-116.808092,33.159927,4055.0,Point,"[-116.8081, 33.1599, 4055]",3072.0,2048.0,http://hpwren.ucsd.edu/cameras/BBlackMtn.html
4,hpwren1_west,west,Big Black Mountain,bm-w-mobo-c,-116.808092,33.159927,4055.0,Point,"[-116.8081, 33.1599, 4055]",3072.0,2048.0,http://hpwren.ucsd.edu/cameras/BBlackMtn.html
...,...,...,...,...,...,...,...,...,...,...,...,...
126,hpwren_missing5_west,west,Santiago Peak,stgo-w-mobo-c,-117.534115,33.711172,5669.0,NaN,NaN,3072.0,2048.0,NaN
127,hpwren_missing6_north,north,White Star,ws-n-mobo-c,-116.318014,32.647266,4000.0,NaN,NaN,3072.0,2048.0,NaN
128,hpwren_missing6_east,east,White Star,ws-e-mobo-c,-116.318014,32.647266,4000.0,NaN,NaN,3072.0,2048.0,NaN
129,hpwren_missing6_south,south,White Star,ws-s-mobo-c,-116.318014,32.647266,4000.0,NaN,NaN,3072.0,2048.0,NaN


# Find Intersections

# Find Landmark Distances

In [3]:
lm_df = pd.read_csv("../data/landmarks_processed.csv")
cam_meta = pd.read_csv("../data/cam_meta_05302022.csv")

lm_df = lm_df.merge(cam_meta[['station', 'dir', 'cam_lat', 'cam_long']], left_on=['station', 'dir'], right_on=['station', 'dir'], how='left')
lm_df = lm_df.drop_duplicates().reset_index(drop=True)
lm_df['distance'] = lm_df.apply(lambda x: haversine((x['lat'], x['long']), (x['cam_lat'], x['cam_long'])), axis=1)
del lm_df['cam_lat']
del lm_df['cam_long']

lm_df['tile'] = lm_df.apply(lambda x: find_tile(x['x_pixel'], x['y_pixel'], img_size = (x['x_res'], x['y_res'])), axis=1)

In [60]:
lm_df

,landmark,station,dir,lat,long,x_pixel,y_pixel,x_res,y_res,intersection,distance,tile
0,white building - southwest,om,s,32.593892,-116.845919,2900,1715,3072,2048,0,0.150093,35
1,white building - southwest,om,w,32.593892,-116.845919,423,1685,3072,2048,0,0.150093,28
2,white building - south,om,s,32.594298,-116.844774,1780,1930,3072,2048,0,0.052190,41
3,communication tower building - east,om,n,32.595487,-116.844141,2545,1725,3072,2048,0,0.095800,34
4,large bush,om,e,32.599053,-116.839254,318,1420,3072,2048,0,0.698102,18
...,...,...,...,...,...,...,...,...,...,...,...,...
127,veritiv distribution warehouse,rdd,n,33.108108,-117.115555,2078,1140,3072,2048,1,1.667744,24
128,mt laguna water tower,mlo,n,32.875975,-116.414988,1966,1055,3072,2048,1,4.177608,14
129,left observatory,mlo,n,32.840830,-116.427060,880,1290,3072,2048,0,0.138275,20
130,tree cluster end,mg,w,33.182096,-116.781314,880,1388,3072,2048,1,2.063995,20


In [4]:
cam_meta

,Unnamed: 0,Unnamed: 0.1,camera_id,camera_name,hpweren_camera_description,cam_lat,cam_long,station,dir,center_lat,center_long,elevation,x_resolution,y_resolution,center_angle
0,0,0,hpwren1_north,bm-n-mobo-c,Big Black Mountain,33.159927,-116.808092,bm,n,33.181599,-116.807554,4055,3072,2048,-0.024816
1,1,1,hpwren1_east,bm-e-mobo-c,Big Black Mountain,33.159927,-116.808092,bm,e,33.158781,-116.790230,4055,3072,2048,-0.064085
2,2,2,hpwren1_south,bm-s-mobo-c,Big Black Mountain,33.159927,-116.808092,bm,s,33.157932,-116.807962,4055,3072,2048,0.065022
3,3,3,hpwren1_west,bm-w-mobo-c,Big Black Mountain,33.159927,-116.808092,bm,w,33.159091,-116.858706,4055,3072,2048,0.016519
4,4,4,hpwren2_north,bl-n-mobo-c,Black Mountain,32.981417,-117.116480,bl,n,32.981788,-117.116520,1600,2048,1536,0.108221
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,125,125,hpwren30_west,marconi-w-mobo-c,Upper Talega,33.519127,-117.480907,marconi,w,33.517974,-117.529087,2300,3072,2048,0.023941
126,126,126,na,ws-n-mobo-c,White Star,32.647266,-116.318014,ws,n,32.650875,-116.317954,4000,3072,2048,-0.016515
127,127,127,na,ws-e-mobo-c,White Star,32.647266,-116.318014,ws,e,32.646405,-116.305941,4000,3072,2048,-0.071166
128,128,128,na,ws-s-mobo-c,White Star,32.647266,-116.318014,ws,s,32.644742,-116.317985,4000,3072,2048,0.011411


In [61]:
lm_df.to_csv("../data/landmarks_processed.csv", index=False)